# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# Import Libraries

In [ ]:
%cd '/content'
!git clone 'https://github.com/vncorenlp/VnCoreNLP.git'
!pip install py_vncorenlp
!pip install -U sentence-transformers

/content
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 259, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 259 (delta 17), reused 14 (delta 2), pack-reused 212
Receiving objects: 100% (259/259), 237.79 MiB | 24.74 MiB/s, done.
Resolving deltas: 100% (93/93), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 12.7 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=908bdc2e7fa23fa91b5cb70fd2027f6c8a8ca85fbfb863892c380ebb23f937e0
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import re
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import gzip
import os
import torch
from nltk import tokenize

# Import Data

In [ ]:
private = pd.read_json('/content/ise-dsc01-private-test-offcial.json')
pridata = np.transpose(private.values)
private_df = pd.DataFrame(pridata, columns = ['context', 'claim'])
private_df.head()

,context,claim
0,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",Xuyên suốt thời gian diễn ra Festival biển Nha...
1,"Còn với Công nghệ dệt may, em không biết may h...",Một số người khuyên Nhi Pham theo Công nghệ th...
2,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,Nhiệt độ da bên dưới lớp cotton cao làm người ...
3,"Trong không gian của du thuyền 5 sao, Shynh Ho...",Nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...
4,Pin sinh học là giải pháp mới để cung cấp năng...,Bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...


# Basic Information

In [ ]:
def basic_inf(df : pd.DataFrame) -> pd.DataFrame:
  basic_dict = {}
  basic_dict['Column'] = list(df.columns)
  basic_dict['Missing value'] = list(df.isna().sum())
  basic_dict['Number of value'] = [len(df[column].unique()) for column in list(df.columns)]
  basic_dict['Values'] = [df[column].unique() for column in list(df.columns)]
  return pd.DataFrame(basic_dict)

In [ ]:
print('Shape: ', private_df.shape)

Shape:  (37967, 5)


In [ ]:
basic_df = basic_inf(train_df)
basic_df

,Column,Missing value,Number of value,Values
0,context,0,4796,"[Phát biểu tại lễ ký kết vào ngày 17/11, Giám ..."
1,claim,0,37211,"[Ngoài việc không giới hạn mức lương, công ty ..."
2,verdict,0,3,"[SUPPORTED, NEI, REFUTED]"
3,evidence,12583,17606,"[Công ty cũng có chế độ đãi ngộ tốt, có lương ..."
4,domain,0,6,"[giao-duc, khoa-hoc, suc-khoe, the-gioi, du-li..."


In [ ]:
#for test
private_df['evidence'] = None

In [ ]:
private_df.head()

,context,claim,verdict,evidence,domain
0,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...",giao-duc
1,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...",giao-duc
2,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...",giao-duc
3,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",giao-duc
4,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...",giao-duc


# Processing Data

In [ ]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 57.9 MB/s eta 0:00:00


In [ ]:
import re
from underthesea import sent_tokenize
def processing_sen(text: str) -> str:
  text = text.lower()
  text = re.sub('\.', '', text)
  text = re.sub('[^\d\w\s/]', ' ', text)
  text = ' '.join(text.split('\n'))
  text = ' '.join(re.split(r'\s+', text))
  text = text.strip()
  return text

def processing_para(para):
  para = re.sub('[^\d\w\s/\.?]', ' ', para)
  para = re.sub('\.+', '.', para)
  corpus = sent_tokenize(para)
  corpus = [processing_sen(sentence) for sentence in corpus]
  corpus = [sentence for sentence in corpus if sentence != '']
  return corpus

In [ ]:
private_df['corpus'] = private_df['context'].apply(processing_para)
private_df['corpus'].head()

0    [sáng 7/6 bà nguyễn thị lệ thanh giám đốc sở d...
1    [còn với công nghệ dệt may em không biết may h...
2    [nhà nghiên cứu guangming tao ở đại học khoa h...
3    [trong không gian của du thuyền 5 sao shynh ho...
4    [pin sinh học là giải pháp mới để cung cấp năn...
Name: corpus, dtype: object

In [ ]:
private_df['claim'] = private_df['claim'].apply(processing_sen)
private_df['claim'].head()

0    xuyên suốt thời gian diễn ra festival biển nha...
1    một số người khuyên nhi pham theo công nghệ th...
2    nhiệt độ da bên dưới lớp cotton cao làm người ...
3    nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...
4    bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...
Name: claim, dtype: object

# Word Segmentation

In [ ]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/content/VnCoreNLP')
# Load the word and sentence segmentation component
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/VnCoreNLP')
text = "Ông Nguyễn Khắc Chúc  đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
output = rdrsegmenter.word_segment(text)
print(output)

VnCoreNLP model folder /content/VnCoreNLP already exists! Please load VnCoreNLP from this folder!
['Ông Nguyễn_Khắc_Chúc đang làm_việc tại Đại_học Quốc_gia Hà_Nội .', 'Bà Lan , vợ ông Chúc , cũng làm_việc tại đây .']


In [ ]:
def word_segmentation_text(text):
  return rdrsegmenter.word_segment(text)[0]
def word_segmentation_corpus(corpus):
  return [word_segmentation_text(text) for text in corpus]

# BM25+SBERT

In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

In [ ]:
model_sim = SentenceTransformer('VoVanPhuc/sup-SimCSE-VietNamese-phobert-base')

In [ ]:
model_keep = SentenceTransformer('keepitreal/vietnamese-sbert')

In [ ]:
import math
def MixTrip_encoder(df,top_k = 3,model1='keepitreal/vietnamese-sbert',model2='VoVanPhuc/sup-SimCSE-VietNamese-phobert-base'):
  model1 = model1
  model2 = model2
  lst_evidence = []
  for _, row in df.iterrows():
    topk=[]
    corpus = row['corpus']
    claim = row['claim']

    seg_corpus = word_segmentation_corpus(corpus)
    seg_claim = word_segmentation_text(claim)

    tokenized_corpus = [doc.split(" ") for doc in seg_corpus]
    bm25 = BM25Okapi(tokenized_corpus)

    tokenized_query = seg_claim.split(" ")
    doc_scores = bm25.get_scores(tokenized_query)
    # for index_sco in range(len(doc_scores)):
    #   doc_scores[index_sco] = math.sqrt(doc_scores[index_sco])

    em_corpus1 = model1.encode(seg_corpus, convert_to_tensor = True)
    em_claim1 = model1.encode(seg_claim, convert_to_tensor = True)
    cosine_scores1 = util.cos_sim(em_corpus1, em_claim1)
    cosine_scores1 = cosine_scores1.cpu().numpy()
    cosine_scores1 = cosine_scores1.reshape(-1)

    em_corpus2 = model2.encode(seg_corpus, convert_to_tensor = True)
    em_claim2 = model2.encode(seg_claim, convert_to_tensor = True)
    cosine_scores2 = util.cos_sim(em_corpus2, em_claim2)
    cosine_scores2 = cosine_scores2.cpu().numpy()
    cosine_scores2 = cosine_scores2.reshape(-1)

    cosine_scores=cosine_scores1*0.8+cosine_scores2*0.2

    scores_index = np.multiply(doc_scores,cosine_scores)
    scores_index = np.argsort(scores_index)
    scores_index = scores_index[::-1]
    for i in range(top_k):
      topk.append(row['corpus'][scores_index[i]])
    top_para = ". ".join(topk)
    lst_evidence.append(top_para)
  return lst_evidence

In [ ]:
%%time
hits_trip = MixTrip_encoder(private_df,1,model_keep,model_sim)
hits_trip

CPU times: user 41min 18s, sys: 4.68 s, total: 41min 23s
Wall time: 41min 12s


['công ty cũng có chế độ đãi ngộ tốt có lương tháng 13 14 phụ cấp',
 'bên cạnh đó faba thường xuyên tổ chức những dự án đào tạo kỹ năng nghề nghiệp kỹ năng mềm chú trọng xây dựng văn hoá doanh nghiệp để thúc đẩy phát triển đội ngũ',
 'bên cạnh đó faba thường xuyên tổ chức những dự án đào tạo kỹ năng nghề nghiệp kỹ năng mềm chú trọng xây dựng văn hoá doanh nghiệp để thúc đẩy phát triển đội ngũ',
 'không riêng kỹ sư phần mềm đối với bất kỳ lĩnh vực nào chính sách đãi ngộ chưa đủ hấp dẫn khó lòng thu hút được nhân tài về nước một tiến sĩ việt kiều nói',
 'theo adeco việt nam mức lương cao nhất đối với vị trí kỹ sư phần mềm hiện nay là 150 triệu đồng/tháng tại hà nội và 160 triệu đồng/tháng tại tp hcm',
 'tiến sĩ liêm cũng ủng hộ phương án vừa làm việc ở nước ngoài vừa hợp tác nghiên cứu với các viện nghiên cứu phòng thí nghiệm nhà khoa học trong nước nhằm tìm giải pháp đào tạo thế hệ trẻ',
 'học sinh này tâm sự rằng các em rất muốn được học nhiều hơn nhưng không dám góp ý với cô vì sợ',
 

In [ ]:
private_df['evidence'] = hits_trip
private_df

,context,claim,corpus,evidence
0,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",xuyên suốt thời gian diễn ra festival biển nha...,[sáng 7/6 bà nguyễn thị lệ thanh giám đốc sở d...,người đứng đầu sở du lịch khánh hòa cho hay xu...
1,"Còn với Công nghệ dệt may, em không biết may h...",một số người khuyên nhi pham theo công nghệ th...,[còn với công nghệ dệt may em không biết may h...,một số người khuyên em theo công nghệ thực phẩ...
2,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,nhiệt độ da bên dưới lớp cotton cao làm người ...,[nhà nghiên cứu guangming tao ở đại học khoa h...,nhiệt độ da bên dưới vải siêu mát tăng từ khoả...
3,"Trong không gian của du thuyền 5 sao, Shynh Ho...",nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...,[trong không gian của du thuyền 5 sao shynh ho...,nhãn hàng sẽ cùng đối tác đề ra những chiến lư...
4,Pin sinh học là giải pháp mới để cung cấp năng...,bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...,[pin sinh học là giải pháp mới để cung cấp năn...,chúng tôi gọi đó là bụi thông minh và chỉ vài ...
...,...,...,...,...
5391,"Mary Kate Keyes, Giám đốc dinh dưỡng của ứng d...",lúc đó tôi đã quyết định về nhà vì tôi cảm thấ...,[mary kate keyes giám đốc dinh dưỡng của ứng d...,ví dụ khi tôi cảm thấy lo lắng và căng thẳng t...
5392,Cô gái Việt đang là nghiên cứu sinh tiến sĩ nă...,ứng viên có thể thông qua mạng lưới bạn bè hoặ...,[cô gái việt đang là nghiên cứu sinh tiến sĩ n...,ứng viên có thể thông qua mạng lưới bạn bè hoặ...
5393,Người bệnh cho biết khoảng một tháng nay ăn uố...,tuy nhiên chỉ số đường huyết của bệnh nhân lúc...,[người bệnh cho biết khoảng một tháng nay ăn u...,tuy nhiên chỉ số đường huyết của bệnh nhân lúc...
5394,Cơ quan Giáo dục New Zealand (ENZ) hôm 5/1 cho...,cơ quan giáo dục new zealand enz hôm 5/1 cho b...,[cơ quan giáo dục new zealand enz hôm 5/1 cho ...,cơ quan giáo dục new zealand enz hôm 5/1 cho b...


In [ ]:
count =0
lst_context=[]
lst_claim=[]
lst_verdict=[]
lst_evidence=[]
lst_domain=[]

for _, row in not_nei.iterrows():
  context = row['context']
  claim = row['claim']
  verdict = row['verdict']
  evidence = row['evidence']
  evidence= "".join(evidence)
  domain = row['domain']
  pred = row['pred-trip']

  if evidence == pred or claim == pred:
    count=count+1
    lst_context.append(context)
    lst_claim.append(claim)
    lst_verdict.append(verdict)
    lst_evidence.append(evidence)
    lst_domain.append(domain)

print("accuracy:",count/not_nei.shape[0])

accuracy: 0.8774818783485661


In [ ]:
import pandas as pd

new = {'context': pd.Series(lst_context),
     'claim': pd.Series(lst_claim),
      'verdict': pd.Series(lst_verdict),
      'evidence': pd.Series(lst_evidence),
       'domain': pd.Series(lst_domain),
       'pred1': pd.Series(lst_pred1),
       'pred2': pd.Series(lst_pred2)}

new_df = pd.DataFrame(new)

<ipython-input-45-812d2ab892f8>:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  'domain': pd.Series(lst_domain),


In [ ]:
new_df

,context,claim,verdict,evidence,domain,pred1,pred2
0,Đứng ngay sau là Viện Công nghệ Massachusetts ...,20 trường đứng đầu về thu nhập của cựu sinh vi...,SUPPORTED,dưới đây là 20 trường đứng đầu về thu nhập của...,NaN,dưới đây là 20 trường đứng đầu về thu nhập của...,hai vị trí xếp trên là đại học harvey mudd và ...
1,"Trần Vũ Minh Tâm, 22 tuổi, vừa trở thành sinh ...",sau khi thay đổi phương pháp dành nhiều thời g...,SUPPORTED,từ một học sinh ở nhóm giữa em vươn lên lọt và...,NaN,từ một học sinh ở nhóm giữa em vươn lên lọt và...,nhưng sau đó em thay đổi phương pháp và dành n...
2,Nhóm nghiên cứu đến từ Tổ chức Khoa học và ngh...,vaccine được bảo quản một cách kỹ lưỡng,SUPPORTED,vật liệu này bao phủ phân tử vaccine bảo vệ ch...,NaN,tuy nhiên vận chuyển vaccine đặc biệt tới các ...,thí nghiệm thành công cho thấy lớp phủ mof bảo...
3,"Pearl chỉ cao 9,14 cm, phá kỷ lục 9,65 cm trướ...",pearl thích ăn diện đẹp đẽ,SUPPORTED,nó cũng thích mặc đẹp,NaN,pearl thích ăn thực phẩm chất lượng cao như th...,nó cũng thích mặc đẹp
4,Kẹp robot được chế tạo dựa trên nghệ thuật cắt...,trong thực tiễn có nhiều trường hợp cần sử dụn...,SUPPORTED,nhóm nghiên cứu cho biết kỹ thuật mới có rất n...,NaN,nhóm nhà khoa học đang nghiên cứu cách tích hợ...,nhóm nghiên cứu cho biết kỹ thuật mới có rất n...
...,...,...,...,...,...,...,...
95,"Lợn hoang, theo cách gọi của Bộ Nông nghiệp Mỹ...",lợn cái có thể sinh sản ở 12 15 tháng tuổi,REFUTED,ngoài ra lợn cái có thể bắt đầu sinh sản ở 8 t...,NaN,cứ cách 12 15 tháng mỗi con lợn cái có thể đẻ ...,ngoài ra lợn cái có thể bắt đầu sinh sản ở 8 t...
96,"Ông Hồ Tấn Minh, Chánh văn phòng Sở Giáo dục v...",nam sinh bị tiếp cận dụ dỗ được cho là đang ở ...,REFUTED,lúc này nam sinh đứng trước cổng trường chờ ng...,NaN,một người đàn ông trung niên chạy xe máy đến t...,theo ông trần công tuấn hiệu trưởng trường thp...
97,"Với khát khao học tập suốt đời, bà Park Kyung-...",bà park luôn tự một mình học chứ không có ai h...,REFUTED,các giáo sư bạn học ở trường cũng hỗ trợ bà vi...,NaN,nếu không có sự hỗ trợ đó tôi khó có thể đạt đ...,các giáo sư bạn học ở trường cũng hỗ trợ bà vi...
98,"Với nhu cầu phát triển đội ngũ nhân lực, xây d...",deepview khuyến khích người dùng xem càng nhiề...,REFUTED,giải pháp này khuyến khích người xem nội dung ...,NaN,điểm độc đáo của deepview là giải pháp sáng ch...,giải pháp này khuyến khích người xem nội dung ...


# ToCSV

In [ ]:
private_df.to_csv('d.csv',encoding='utf-8-sig',escapechar="|")